In [39]:
import json

# Load data from JSON
with open('apiResponse/all_responses_200_sample.json', 'r', encoding='utf-8') as file:
    api_data = json.load(file)

with open('Samples/sample_siirtokarjalaiset_annotated.json', 'r', encoding='utf-8') as file:
    hand_data = json.load(file)



In [46]:

from fuzzywuzzy import fuzz

def parse_response(response_str):
    lines = response_str.split('\n')
    parsed_response = {}
    for line in lines:
        key, _, value = line.partition(': ')
        parsed_response[key] = value.strip() if value.strip() else None
    return parsed_response


# Checking how similar the words are
def are_similar(str1, str2, threshold=70, context=None):
    similarity = fuzz.token_set_ratio(str1, str2)
    is_similar = similarity > threshold
    
    # Check if similarity is below 100% and store it if so
    if is_similar and similarity < 100:
        store_not_exact_matches(str1, str2, similarity, context)
        
    return is_similar


def store_not_exact_matches(str1, str2, similarity, context):
    data = {
        "string_1": str1,
        "string_2": str2,
        "similarity": similarity,
        "context": context  # This will provide additional information
    }
    with open("non_exact_matches.json", "a", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)
        file.write(",\n")






In [47]:
def compare_values(api_values, annotated_values, context):
    api_list = api_values.lower().split(', ') if api_values else []
    annotated_list = annotated_values.lower().split(', ') if annotated_values else []
    
    matches = set()
    mismatches = set(api_list).union(set(annotated_list))  
    
    for api_val in api_list:
        for ann_val in annotated_list:
            if are_similar(api_val, ann_val, context=context):
                matches.add(api_val)
                mismatches.discard(api_val)
                mismatches.discard(ann_val)
    
    return matches, mismatches


# Parse JSON strings
api_responses = api_data
hand_annotated = hand_data
# Loop over all elements in api_responses and hand_annotated to compare them
results = []
total_matches = 0
total_mismatches = 0

for api_resp, hand_ann in zip(api_responses, hand_annotated):
    parsed_api_response = parse_response(api_resp['api_response'])
    
    comparison_results = {
        "index": hand_ann['index'],
        "person_name": hand_ann['primary_person_name'],
        "spouse_name": hand_ann['spouse_name'],
        "detail": []
    }
    
    for key in ["person_hobbies", "person_social_orgs", "spouse_hobbies", "spouse_social_orgs"]:
        split_keys = key.split("_")
        api_key = split_keys[0].capitalize() + "".join(word.capitalize() for word in split_keys[1:])
        
        # Safely get the index values for context
        api_person_index = api_resp.get('person_index', None)
        annotated_person_index = hand_ann.get('index', None)
        

        # Build the context
        context = {
            "api_person_index": api_person_index +1,
            "annotated_person_index": annotated_person_index,
            "category_type": key
        }
        
        matches, mismatches = compare_values(parsed_api_response.get(api_key, ""), hand_ann[key], context)
        
        detail = {
            "type": key,
            "matches": list(matches),
            "mismatches": list(mismatches)
        }
        comparison_results["detail"].append(detail)
        
        total_matches += len(matches)
        total_mismatches += len(mismatches)
    
    results.append(comparison_results)

output_json = json.dumps(results, indent=4, ensure_ascii=False)

# To store the results in a file:
with open("matches_results.json", "w") as file:
    file.write(output_json)

# Printing total matches and mismatches
print(f"Total Matches: {total_matches}")
print(f"Total Mismatches: {total_mismatches}")

# Calculating and printing the match percentage
total_comparisons = total_matches + total_mismatches
if total_comparisons > 0: 
    match_percentage = (total_matches / total_comparisons) * 100
    print(f"Match Percentage: {match_percentage:.2f}%")
else:
    print("No comparisons were made (Total Comparisons: 0).")

Total Matches: 350
Total Mismatches: 486
Match Percentage: 41.87%


In [48]:
### Precision, recall and F Score 

def compare_values(api_dict, hand_ann_dict):
    api_name = api_dict["api_response"].split("PersonName:")[1].split("\n")[0].strip()
    api_hobbies = api_dict["api_response"].split("PersonHobbies:")[1].split("\n")[0].strip().split(', ')
    api_social_orgs = api_dict["api_response"].split("PersonSocialOrgs:")[1].split("\n")[0].strip().split(', ')
    api_spouse_hobbies = api_dict["api_response"].split("SpouseHobbies:")[1].split("\n")[0].strip().split(', ')
    api_spouse_social_orgs = api_dict["api_response"].split("SpouseSocialOrgs:")[1].split("\n")[0].strip().split(', ')

    hand_name = hand_ann_dict["primary_person_name"]
    hand_hobbies = hand_ann_dict["person_hobbies"].split(', ')
    hand_social_orgs = hand_ann_dict["person_social_orgs"].split(', ')
    hand_spouse_hobbies = hand_ann_dict["spouse_hobbies"].split(', ')
    hand_spouse_social_orgs = hand_ann_dict["spouse_social_orgs"].split(', ')

    # Assuming are_similar function is predefined

    def calculate_metrics(api_values, hand_values):
        TP = set()
        FP = set(api_values)  # Temporarily assume all api_values are FP
        FN = set(hand_values)  # Temporarily assume all annotated_values are FN
        
        for api_val in api_values:
            for ann_val in hand_values:
                if are_similar(api_val, ann_val):
                    TP.add(api_val)  # Add to True Positives
                    FP.discard(api_val)  # Remove from False Positives
                    FN.discard(ann_val)  # Remove from False Negatives
                    
        precision = len(TP) / (len(TP) + len(FP)) if TP or FP else 0
        recall = len(TP) / (len(TP) + len(FN)) if TP or FN else 0
        f_score = (2 * precision * recall) / (precision + recall) if precision + recall > 0 else 0
    
        return TP, FP, FN, precision, recall, f_score

    # Compute metrics for person and spouse separately
    person_metrics = calculate_metrics(api_social_orgs, hand_social_orgs)
    person_hobbies = calculate_metrics(api_hobbies, hand_hobbies)
    spouse_metrics = calculate_metrics(api_spouse_social_orgs, hand_spouse_social_orgs)
    spouse_hobbies = calculate_metrics(api_spouse_hobbies, hand_spouse_hobbies)

    

    return person_metrics, spouse_metrics, person_hobbies, spouse_hobbies

    



api_responses = api_data  # This should be your API data
hand_annotated = hand_data  # This should be your hand annotated data

results = []


# Assume api_responses and hand_annotated are lists of strings for this example. 
# Modify as per your actual data structure.
total_person_TP = total_person_FP = total_person_FN = 0
total_spouse_TP = total_spouse_FP = total_spouse_FN = 0
total_person_hobbies_TP = total_person_hobbies_FP = total_person_hobbies_FN = 0
total_spouse_hobbies_TP = total_spouse_hobbies_FP = total_spouse_hobbies_FN = 0


for index, (api_resp, hand_ann) in enumerate(zip(api_responses, hand_annotated)):
    # Call compare_values and unpack all four return values here:
    person_metrics, spouse_metrics, person_hobbies_metrics, spouse_hobbies_metrics = compare_values(api_resp, hand_ann)

    total_person_TP += len(person_metrics[0])
    total_person_FP += len(person_metrics[1])
    total_person_FN += len(person_metrics[2])

    total_spouse_TP += len(spouse_metrics[0])
    total_spouse_FP += len(spouse_metrics[1])
    total_spouse_FN += len(spouse_metrics[2])
    
    # Add handling for person_hobbies_metrics and spouse_hobbies_metrics
    # For example:
    total_person_hobbies_TP += len(person_hobbies_metrics[0])
    total_person_hobbies_FP += len(person_hobbies_metrics[1])
    total_person_hobbies_FN += len(person_hobbies_metrics[2])

    total_spouse_hobbies_TP += len(spouse_hobbies_metrics[0])
    total_spouse_hobbies_FP += len(spouse_hobbies_metrics[1])
    total_spouse_hobbies_FN += len(spouse_hobbies_metrics[2])

    results.append({
    "person_metrics": {
        "index": hand_ann['index'],
        "person_name": hand_ann['primary_person_name'],
        "true_positives": list(person_metrics[0]),
        "false_positives": list(person_metrics[1]),
        "false_negatives": list(person_metrics[2]),
        "precision": person_metrics[3],
        "recall": person_metrics[4],
        "f_score": person_metrics[5]
    },
    "person_hobbies_metrics": {
        "true_positives": list(person_hobbies_metrics[0]),
        "false_positives": list(person_hobbies_metrics[1]),
        "false_negatives": list(person_hobbies_metrics[2]),
        "precision": person_hobbies_metrics[3],
        "recall": person_hobbies_metrics[4],
        "f_score": person_hobbies_metrics[5]
    },
    "spouse_metrics": {
        "true_positives": list(spouse_metrics[0]),
        "false_positives": list(spouse_metrics[1]),
        "false_negatives": list(spouse_metrics[2]),
        "precision": spouse_metrics[3],
        "recall": spouse_metrics[4],
        "f_score": spouse_metrics[5]
    },
    "spouse_hobbies_metrics": {
        "true_positives": list(spouse_hobbies_metrics[0]),
        "false_positives": list(spouse_hobbies_metrics[1]),
        "false_negatives": list(spouse_hobbies_metrics[2]),
        "precision": spouse_hobbies_metrics[3],
        "recall": spouse_hobbies_metrics[4],
        "f_score": spouse_hobbies_metrics[5]
    }
})


# Constructing a summary of results for final output
person_precision = total_person_TP / (total_person_TP + total_person_FP) if total_person_TP + total_person_FP > 0 else 0
person_recall = total_person_TP / (total_person_TP + total_person_FN) if total_person_TP + total_person_FN > 0 else 0
spouse_precision = total_spouse_TP / (total_spouse_TP + total_spouse_FP) if total_spouse_TP + total_spouse_FP > 0 else 0
spouse_recall = total_spouse_TP / (total_spouse_TP + total_spouse_FN) if total_spouse_TP + total_spouse_FN > 0 else 0

# Calculate overall metrics
overall_TP = total_person_TP + total_spouse_TP
overall_FP = total_person_FP + total_spouse_FP
overall_FN = total_person_FN + total_spouse_FN

overall_precision = overall_TP / (overall_TP + overall_FP) if (overall_TP + overall_FP) > 0 else 0
overall_recall = overall_TP / (overall_TP + overall_FN) if (overall_TP + overall_FN) > 0 else 0
overall_f_score = (2 * overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0

# Round to 3 decimal places
overall_precision = round(overall_precision, 3)
overall_recall = round(overall_recall, 3)
overall_f_score = round(overall_f_score, 3)

summary = {
    "person": {
        "total_true_positives": total_person_TP,
        "total_false_positives": total_person_FP,
        "total_false_negatives": total_person_FN,
        "precision": person_precision,
        "recall": person_recall,
        "f_score": (2 * person_precision * person_recall) / (person_precision + person_recall) if person_precision + person_recall > 0 else 0
    },
    "spouse": {
        "total_true_positives": total_spouse_TP,
        "total_false_positives": total_spouse_FP,
        "total_false_negatives": total_spouse_FN,
        "precision": spouse_precision,
        "recall": spouse_recall,
        "f_score": (2 * spouse_precision * spouse_recall) / (spouse_precision + spouse_recall) if spouse_precision + spouse_recall > 0 else 0
    }, 
    "overall": {
        "total_true_positives": overall_TP,
        "total_false_positives": overall_FP,
        "total_false_negatives": overall_FN,
        "precision": overall_precision,
        "recall": overall_recall,
        "f_score": overall_f_score
    }
}
for key in summary:
    summary[key]['precision'] = round(summary[key]['precision'], 3)
    summary[key]['recall'] = round(summary[key]['recall'], 3)
    summary[key]['f_score'] = round(summary[key]['f_score'], 3)


# Final JSON output to include both the detailed results and the summary
output_data = {
    "summary": summary,
    "results": results
}

# Storing in JSON format
output_json = json.dumps(output_data, indent=4, ensure_ascii=False)

# To store the results in a file:
with open("precision_recall_results.json", "w", encoding="utf-8") as file:
    file.write(output_json)

# Printing total matches and mismatches
print(f"--- Person ---")
print(f"Total True Positives: {total_person_TP}")
print(f"Total False Positives: {total_person_FP}")
print(f"Total False Negatives: {total_person_FN}")
print(f"Precision: {summary['person']['precision']:.2f}")
print(f"Recall: {summary['person']['recall']:.2f}")
print(f"\n--- Spouse ---")
print(f"Total True Positives: {total_spouse_TP}")
print(f"Total False Positives: {total_spouse_FP}")
print(f"Total False Negatives: {total_spouse_FN}")
print(f"Precision: {summary['spouse']['precision']:.2f}")
print(f"Recall: {summary['spouse']['recall']:.2f}")
print(f"\n--- Overall ---")
print(f"Total True Positives: {overall_TP}")
print(f"Total False Positives: {overall_FP}")
print(f"Total False Negatives: {overall_FN}")
print(f"Precision: {overall_precision:.3f}")
print(f"Recall: {overall_recall:.3f}")
print(f"F-score: {overall_f_score:.3f}")

--- Person ---
Total True Positives: 128
Total False Positives: 161
Total False Negatives: 194
Precision: 0.44
Recall: 0.40

--- Spouse ---
Total True Positives: 59
Total False Positives: 193
Total False Negatives: 213
Precision: 0.23
Recall: 0.22

--- Overall ---
Total True Positives: 187
Total False Positives: 354
Total False Negatives: 407
Precision: 0.346
Recall: 0.315
F-score: 0.330


In [49]:
## Evaluation for true negative
## Basicly adding emptylabel in true positive if there is no false positive or false negative

# Load the JSON data
with open('precision_recall_results.json', 'r') as file:
    data = json.load(file)

# Iterate through the "results" section
for result in data['results']:
    for metric_key, metric_value in result.items():
        
        # Check if all entities are empty
        if not any(metric_value['true_positives']) and \
           not any(metric_value['false_positives']) and \
           not any(metric_value['false_negatives']):
            metric_value['true_positives'] = ["emptylabel"]
            metric_value['f_score'] = 1.0
            metric_value['precision'] = 1.0
            metric_value['recall'] = 1.0
        else:
            # Calculate precision, recall, and F-score
            try:
                metric_value['precision'] = len(metric_value['true_positives']) / \
                                            (len(metric_value['true_positives']) + len(metric_value['false_positives']))
            except ZeroDivisionError:
                metric_value['precision'] = 0.0
                
            try:
                metric_value['recall'] = len(metric_value['true_positives']) / \
                                         (len(metric_value['true_positives']) + len(metric_value['false_negatives']))
            except ZeroDivisionError:
                metric_value['recall'] = 0.0
                
            try:
                metric_value['f_score'] = 2 * (metric_value['precision'] * metric_value['recall']) / \
                                          (metric_value['precision'] + metric_value['recall'])
            except ZeroDivisionError:
                metric_value['f_score'] = 0.0

# Save the updated data back to the JSON file
with open('precision_recall_updated_results.json', 'w') as file:
    json.dump(data, file, indent=4, ensure_ascii=False)

In [50]:
## calculating for summary for all categories
#micro f score


def calculate_metrics(TP, FP, FN):
    try:
        precision = round((TP / (TP + FP)) if TP + FP != 0 else 0, 2)
    except ZeroDivisionError:
        precision = 0.0
        
    try:
        recall = round((TP / (TP + FN)) if TP + FN != 0 else 0, 2)
    except ZeroDivisionError:
        recall = 0.0
        
    try:
        f_score = round((2 * precision * recall) / (precision + recall) if precision + recall != 0 else 0, 2)
    except ZeroDivisionError:
        f_score = 0.0
    
    return precision, recall, f_score

# Load the JSON data
with open('precision_recall_updated_results.json', 'r') as file:
    data = json.load(file)

# Initialize metrics counters
metrics_counter = {
    "person": {"TP": 0, "FP": 0, "FN": 0},
    "spouse": {"TP": 0, "FP": 0, "FN": 0},
    "overall": {"TP": 0, "FP": 0, "FN": 0},
    "social_orgs": {"TP": 0, "FP": 0, "FN": 0},
    "hobbies": {"TP": 0, "FP": 0, "FN": 0},
    "person_social_orgs": {"TP": 0, "FP": 0, "FN": 0},
    "person_hobbies": {"TP": 0, "FP": 0, "FN": 0},
    "spouse_social_orgs": {"TP": 0, "FP": 0, "FN": 0},
    "spouse_hobbies": {"TP": 0, "FP": 0, "FN": 0},
}

# Iterate through the "results" section
for result in data['results']:
    for key, value in result.items():
        if "person_metrics" in key:
            # adding persons social orgs to person_social_orgs
            metrics_counter["person_social_orgs"]["TP"] += len(value["true_positives"])
            metrics_counter["person_social_orgs"]["FP"] += len(value["false_positives"])
            metrics_counter["person_social_orgs"]["FN"] += len(value["false_negatives"])
            # adding social orgs into person
            metrics_counter["person"]["TP"] += len(value["true_positives"])
            metrics_counter["person"]["FP"] += len(value["false_positives"])
            metrics_counter["person"]["FN"] += len(value["false_negatives"])
            # adding person social orgs into social orgs
            metrics_counter["social_orgs"]["TP"] += len(value["true_positives"])
            metrics_counter["social_orgs"]["FP"] += len(value["false_positives"])
            metrics_counter["social_orgs"]["FN"] += len(value["false_negatives"])

        if "person_hobbies_metrics" in key:
            #adding person hobbies into person hobbies
            metrics_counter["person_hobbies"]["TP"] += len(value["true_positives"])
            metrics_counter["person_hobbies"]["FP"] += len(value["false_positives"])
            metrics_counter["person_hobbies"]["FN"] += len(value["false_negatives"])            
            #adding person hobbies into person
            metrics_counter["person"]["TP"] += len(value["true_positives"])
            metrics_counter["person"]["FP"] += len(value["false_positives"])
            metrics_counter["person"]["FN"] += len(value["false_negatives"])
            #adding person hobbies into hobbies
            metrics_counter["hobbies"]["TP"] += len(value["true_positives"])
            metrics_counter["hobbies"]["FP"] += len(value["false_positives"])
            metrics_counter["hobbies"]["FN"] += len(value["false_negatives"])


        if "spouse_metrics" in key:
            #adding spouse social orgs into spouse social orgs
            metrics_counter["spouse_social_orgs"]["TP"] += len(value["true_positives"])
            metrics_counter["spouse_social_orgs"]["FP"] += len(value["false_positives"])
            metrics_counter["spouse_social_orgs"]["FN"] += len(value["false_negatives"])           
            #adding spouse social orgs into spouse
            metrics_counter["spouse"]["TP"] += len(value["true_positives"])
            metrics_counter["spouse"]["FP"] += len(value["false_positives"])
            metrics_counter["spouse"]["FN"] += len(value["false_negatives"])
            #adding spouse social orgs into social orgs
            metrics_counter["social_orgs"]["TP"] += len(value["true_positives"])
            metrics_counter["social_orgs"]["FP"] += len(value["false_positives"])
            metrics_counter["social_orgs"]["FN"] += len(value["false_negatives"])

        if "spouse_hobbies_metrics" in key:
            #adding spouse hobbies into spouse hobbies
            metrics_counter["spouse_hobbies"]["TP"] += len(value["true_positives"])
            metrics_counter["spouse_hobbies"]["FP"] += len(value["false_positives"])
            metrics_counter["spouse_hobbies"]["FN"] += len(value["false_negatives"])
            #adding spouse hobbies into spouse
            metrics_counter["spouse"]["TP"] += len(value["true_positives"])
            metrics_counter["spouse"]["FP"] += len(value["false_positives"])
            metrics_counter["spouse"]["FN"] += len(value["false_negatives"])
            #adding spouse hobbies into hobbies
            metrics_counter["hobbies"]["TP"] += len(value["true_positives"])
            metrics_counter["hobbies"]["FP"] += len(value["false_positives"])
            metrics_counter["hobbies"]["FN"] += len(value["false_negatives"])


        # Update overall metrics
        metrics_counter["overall"]["TP"] += len(value["true_positives"])
        metrics_counter["overall"]["FP"] += len(value["false_positives"])
        metrics_counter["overall"]["FN"] += len(value["false_negatives"])

# Update summary metrics in data
for category, metrics in metrics_counter.items():
    precision, recall, f_score = calculate_metrics(metrics["TP"], metrics["FP"], metrics["FN"])
    data["summary"][category] = {
        "total_true_positives": metrics["TP"],
        "total_false_positives": metrics["FP"],
        "total_false_negatives": metrics["FN"],
        "precision": precision,
        "recall": recall,
        "f_score": f_score
    }

# Save the updated data back to the JSON file
with open('precision_recall_updated_results.json', 'w') as file:
    json.dump(data, file, indent=4, ensure_ascii=False)

In [51]:
## calculating for summary for all categories
#macro f score

# Load the JSON data
with open('precision_recall_updated_results.json', 'r') as file:
    data = json.load(file)

averages = []

for result in data["results"]:
    precision_sum = 0.0
    recall_sum = 0.0
    f_score_sum = 0.0
    
    entities = ["person_metrics", "person_hobbies_metrics", "spouse_metrics", "spouse_hobbies_metrics"]
    num_entities = len(entities)
    
    for entity in entities:
        precision_sum += result[entity]["precision"]
        recall_sum += result[entity]["recall"]
        f_score_sum += result[entity]["f_score"]
    
    avg_precision = precision_sum / num_entities
    avg_recall = recall_sum / num_entities
    avg_f_score = f_score_sum / num_entities
    
    averages.append({
        "average_precision": avg_precision,
        "average_recall": avg_recall,
        "average_f_score": avg_f_score
    })

# Now `averages` contains the average precision, recall, and F-score
# for each person in the original "results" array.

total_precision = 0.0
total_recall = 0.0
total_f_score = 0.0
num_instances = len(averages)

# Summing up all the average scores
for avg in averages:
    total_precision += avg['average_precision']
    total_recall += avg['average_recall']
    total_f_score += avg['average_f_score']

# Calculating the macro-average for all instances
macro_avg_precision = total_precision / num_instances
macro_avg_recall = total_recall / num_instances
macro_avg_f_score = total_f_score / num_instances

micro_avg_precision = round((metrics_counter["overall"]["TP"] / (metrics_counter["overall"]["TP"] + metrics_counter["overall"]["FP"])) if metrics_counter["overall"]["TP"] + metrics_counter["overall"]["FP"] != 0 else 0, 2)
micro_avg_recall = round((metrics_counter["overall"]["TP"] / (metrics_counter["overall"]["TP"] + metrics_counter["overall"]["FN"])) if metrics_counter["overall"]["TP"] + metrics_counter["overall"]["FN"] != 0 else 0, 2)
micro_avg_f_score = round((2 * micro_avg_precision * micro_avg_recall) / (micro_avg_precision + micro_avg_recall) if micro_avg_precision + micro_avg_recall != 0 else 0, 2)

data["summary"]["micro_average"] = {
    "total_true_positives": metrics_counter["overall"]["TP"],
    "total_false_positives": metrics_counter["overall"]["FP"],
    "total_false_negatives": metrics_counter["overall"]["FN"],
    "precision": micro_avg_precision,
    "recall": micro_avg_recall,
    "f_score": micro_avg_f_score
}

# Add macro averages to the summary
data["summary"]["macro_average"] = {
    "total_true_positives": None,  # These values are not defined in macro-averaging
    "total_false_positives": None,  # These values are not defined in macro-averaging
    "total_false_negatives": None,  # These values are not defined in macro-averaging
    "precision": macro_avg_precision,
    "recall": macro_avg_recall,
    "f_score": macro_avg_f_score
}

# Save the updated data back to the JSON file
with open('precision_recall_updated_results.json', 'w') as file:
    json.dump(data, file, indent=4, ensure_ascii=False)

# Print Macro and Micro averages
print(f"Macro-average Precision: {macro_avg_precision:.2f}")
print(f"Macro-average Recall: {macro_avg_recall:.2f}")
print(f"Macro-average F-score: {macro_avg_f_score:.2f}")
print("--")
print(f"Micro-average Precision: {data['summary']['micro_average']['precision']:.2f}")
print(f"Micro-average Recall: {data['summary']['micro_average']['recall']:.2f}")
print(f"Micro-average F-score: {data['summary']['micro_average']['f_score']:.2f}")
print("----------")
for i, avg in enumerate(averages):
   print(f"Person {i+1} - Average Precision: {avg['average_precision']:.2f}, Average Recall: {avg['average_recall']:.2f}, Average F-score: {avg['average_f_score']:.2f}")


Macro-average Precision: 0.71
Macro-average Recall: 0.71
Macro-average F-score: 0.71
--
Micro-average Precision: 0.52
Micro-average Recall: 0.48
Micro-average F-score: 0.50
----------
Person 1 - Average Precision: 0.75, Average Recall: 0.75, Average F-score: 0.75
Person 2 - Average Precision: 1.00, Average Recall: 0.92, Average F-score: 0.95
Person 3 - Average Precision: 1.00, Average Recall: 1.00, Average F-score: 1.00
Person 4 - Average Precision: 0.75, Average Recall: 0.75, Average F-score: 0.75
Person 5 - Average Precision: 0.00, Average Recall: 0.00, Average F-score: 0.00
Person 6 - Average Precision: 1.00, Average Recall: 1.00, Average F-score: 1.00
Person 7 - Average Precision: 0.40, Average Recall: 0.50, Average F-score: 0.44
Person 8 - Average Precision: 0.25, Average Recall: 0.25, Average F-score: 0.25
Person 9 - Average Precision: 1.00, Average Recall: 1.00, Average F-score: 1.00
Person 10 - Average Precision: 1.00, Average Recall: 1.00, Average F-score: 1.00
Person 11 - Ave